In [22]:
import requests

In [23]:
import geopandas as gpd
import pandas as pd

## Open State data

In [37]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Asian Data

In [44]:
table = "B02019"

url = "https://api.census.gov/data/2023/acs/acs5"
params = {
    "get": f"group({table})",
    "ucgid": "pseudo(0100000US$0400000)",
}
response = requests.get(url, params=params)

In [45]:
data = response.json()
columns = data[0]
rows = data[1:]
pi_df = pd.DataFrame(rows, columns=columns)

In [46]:
url = "https://api.census.gov/data/2023/acs/acs5/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [47]:
b02019_vars = {k: v["label"] for k, v in variables.items() if k.startswith("B02019")}
b02019_vars["ucgid"] = "AFFGEOID"

In [48]:
pi_df = pi_df.rename(columns=b02019_vars)
pi_df = pi_df[list(b02019_vars.values())]

In [55]:
rename_dict = {
    col: col.split("!!")[-1]
    for col in pi_df.columns
    if col.startswith("Estimate!!Total")
}
pi_df.rename(columns=rename_dict, inplace=True)

In [56]:
ethnicity_cols = list(rename_dict.values())
pi_df[ethnicity_cols] = pi_df[ethnicity_cols].astype(int)

In [59]:
ethnicity_cols.remove("Total Groups Tallied:")

In [60]:
pi_df["ancestry_countries"] = pi_df[ethnicity_cols].idxmax(axis=1)

## Merge Data

In [63]:
pi_gdf = states_df.merge(pi_df, on="AFFGEOID", how="inner")

In [64]:
pi_gdf = pi_gdf.to_crs(9311)
pi_gdf.to_file("data/Largest_Pacific_Islander_Subgroup_Per_State.gpkg")

In [65]:
pi_gdf.groupby("ancestry_countries").size()

ancestry_countries
Marshallese                                                         1
Native Hawaiian                                                    31
Other Micronesian                                                   1
Other Native Hawaiian and Other Pacific Islander, not specified    17
Samoan                                                              2
dtype: int64